In [1]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from utils import geo
import pandas as pd
import regionmask
from dask.diagnostics import ProgressBar

In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': False})

In [3]:
rootdir = '/local/data/globcolour/from_cmems/OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_082/SouthernOcean_30Sto65S/daily/in_months/'
filename = '*.nc'
def preprocess(ds):
    return ds.where(ds['time.month']==ds['time.month'][0],drop=True).chunk({'time':1})
ds = xr.open_mfdataset(rootdir+filename,preprocess=preprocess)
ds = ds.sel(lat=slice(-40,-70))
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,periodic='X')
ds['area'] = ds['dxC']*ds['dyC']

In [4]:
# Apply a landmask
mask = regionmask.defined_regions.natural_earth.land_110.mask(ds['lon'], ds['lat'])
mask = mask.where(np.isfinite(mask),1)
mask = mask.where(mask==1,np.nan)
# mask.plot()
ds['CHL'] = ds['CHL'].fillna(0)*mask

In [26]:
variable='CHL'
# Extract spatial average for each year
years = np.arange(1998,2022)
days = np.arange(0,365)
# Ordering such that years run from July to June
ds_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = xr.cftime_range(start,end)
    x = (ds[variable].sel({'time':slice(start,end)})).weighted(ds['area'].fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        ds_year.loc[{'year':year}]=x.values
ds_year.to_netcdf('../../data/globcolour_cmems_daily_'+variable+'_wmean-latlon_byyear-JULtoJUN_new.nc')

1998
[########################################] | 100% Completed | 43.5s
1999
[########################################] | 100% Completed | 49.4s
2000
[########################################] | 100% Completed | 57.5s
2001
[########################################] | 100% Completed |  1min  3.0s
2002
[########################################] | 100% Completed |  1min  1.5s
2003
[########################################] | 100% Completed |  1min  1.8s
2004
[########################################] | 100% Completed |  1min  2.6s
2005
[########################################] | 100% Completed |  1min  4.3s
2006
[########################################] | 100% Completed |  1min  2.0s
2007
[########################################] | 100% Completed |  1min  0.9s
2008
[########################################] | 100% Completed |  1min  6.9s
2009
[########################################] | 100% Completed |  1min  8.2s
2010
[########################################] | 100% Completed |  1m

### By basin

In [5]:
cond_atlantic = (ds['lon']>=-70) & (ds['lon']<20)
cond_indian = (ds['lon']>=20) & (ds['lon']<150)
cond_pacific = (ds['lon']>=150) | (ds['lon']<-70)
atlantic = ds['CHL'].where(cond_atlantic,drop=True)
atlantic_area = ds['area'].where(cond_atlantic,drop=True)
indian = ds['CHL'].where(cond_indian,drop=True)
indian_area = ds['area'].where(cond_indian,drop=True)
pacific = ds['CHL'].where(cond_pacific,drop=True)
pacific_area = ds['area'].where(cond_pacific,drop=True)

In [8]:
# Extract spatial average for each year
years = np.arange(1998,2022)
days = np.arange(0,365)
# Ordering such that years run from July to June
atlantic_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
indian_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
pacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = xr.cftime_range(start,end)
    
    # Atlantic
    print('atlantic')
    x = (atlantic.sel({'time':slice(start,end)})).weighted(atlantic_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        atlantic_year.loc[{'year':year}]=x.values
        
    # Indian
    print('indian')
    x = (indian.sel({'time':slice(start,end)})).weighted(indian_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        indian_year.loc[{'year':year}]=x.values
        
    # Pacific
    print('pacific')
    x = (pacific.sel({'time':slice(start,end)})).weighted(pacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        pacific_year.loc[{'year':year}]=x.values

atlantic_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_atlantic.nc')
indian_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_indian.nc')
pacific_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_pacific.nc')

1998
atlantic
[########################################] | 100% Completed | 18.8s
indian
[########################################] | 100% Completed | 19.1s
pacific
[########################################] | 100% Completed | 19.7s
1999
atlantic
[########################################] | 100% Completed | 23.3s
indian
[########################################] | 100% Completed | 24.0s
pacific
[########################################] | 100% Completed | 24.1s
2000
atlantic
[########################################] | 100% Completed | 26.2s
indian
[########################################] | 100% Completed | 27.5s
pacific
[########################################] | 100% Completed | 25.4s
2001
atlantic
[########################################] | 100% Completed | 28.7s
indian
[########################################] | 100% Completed | 31.8s
pacific
[########################################] | 100% Completed | 30.8s
2002
atlantic
[########################################] | 100% Comp

NameError: name 'variable' is not defined

In [9]:
atlantic_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_atlantic.nc')
indian_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_indian.nc')
pacific_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_pacific.nc')